# Discussion

This notebook discusses the paper titled **"Regularizing and Optimizing LSTM Language Models"** by *Merity et al* that was published in 2017.  

### Introduction

Recurrent Neural Networks and their variations are very likely to overfit the training data. This is due to the large network formed by unfolding each cell of the RNN, and *relatively* small number of parameters (since they are shared over each time step) and training data. Thus, the perplexities obtained on the test data are often quite larger than expected. Several attempts have been made to minimize this problem using varied **regularization** techniques. This paper tackles this issue by proposing a model that combines several of such existing methods.

*Merity et al*'s model is a modification of the standard **LSTM** in which *DropConnect* is applied to the hidden weights in the *recurrent* connections of the LSTM for regularization. The dropout mask for each weight is preserved and the same mask is used across all time steps, thereby adding negligible computation overhead. Apart from this, several other techniques have been incorporated :
* **Variational dropout** : The same dropout mask is used for a particular recurrent connection in both the forward and backward pass for all time steps. Each input of a mini-batch has a separate dropout mask, which ensures that the regularizing effect due to it isn't identical across different inputs.
* **Embedding dropout** : Dropout with dropout probability $p_e$ is applied to word embedding vectors, which results in new word vectors which are identically zero for the dropped words. The remaining word vectors are scaled by $\frac{1}{1-p_e}$ as compensation.
* **AR and TAR** : AR (Activation Regularization) and TAR (Temporal Activation Regularization) are modifications of $L_2$ regularization, wherein the standard technique is applied to dropped *output activations* and dropped *change in output activations* respectively. Mathematically, the additional terms in the cost function $J$ are (here $\alpha$ and $\beta$ are scaling constants and $\textbf{D}$ is the dropout mask) :

$$
J_{AR}=\alpha L_2\left(\textbf{D}_l^t\odot h_l^t\right)\\
J_{TAR}=\beta L_2\left(\textbf{D}_l^t\odot\left(h_l^t - h_l^{t-1}\right)\right)
$$
    
* **Weight tying** : In this method, the parameters for word embeddings and the final output layer are shared.
* **Variable backpropagation steps** : A random number of BPTT steps are taken instead of a fixed number, whose mean is very close to the original fixed value ($s$). The BPTT step-size ($x$) is drawn from the following distribution (here $\mathcal{N}$ is the Gaussian distribution, $p$ is a number close to 0.95 and $\sigma^2$ is the desired variance) :
$$
x \sim p\cdot \mathcal{N}\left(s,\sigma^2\right) + (1-p)\cdot \mathcal{N}\left(\frac{s}{2},\sigma^2\right)
$$

* **Independent sizes of word embeddings and hidden layer** : The sizes of the hidden layer and word embeddings are kept independent of each other.

The paper also introduces a new optimization algorithm, namely **Non-monotonically Triggered Averaged Stochastic Gradient Descent** or NT-ASGD, which can be programmatically described as follows :
```python
def NT_ASGD(f, w0, n, L):
    """ Input parameters :
    f - the function to be minimized
    w0 - initial parameters
    n - window size
```

### Network description

### Result highlights

# Implementation
The findings of this paper have been implemented using TensorFlow 1.4 and Numpy 1.13.3, and the results obtained for PTB and Brown datasets have been described later.

### Results